In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('../input/titanic/train.csv')


In [ ]:
X_test=pd.read_csv('../input/titanic/test.csv')
y_test=pd.read_csv('../input/titanic/gender_submission.csv')

We have already read the data and now its time for data preprocessing.Looking at the various mean values and also finding if there is any null value in our dataset and various other steps .


In [ ]:
train.describe()

In [ ]:
train.isnull()

From here we can see that there are null values in our Cabin and Age column but a better way to see this is by plotting heatmap using seaborn lib or by just getting info about the datset.

In [ ]:
train.info()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.isnull(),yticklabels=False,cbar=True)


From here we can see that there are null values in **Embarked column** as well and we might want to remove this null values for a better result.

So lets deal with the Age dataset first and try to replace these NaN values with some values that can do well.One way to do so is to just replace the values with the mean of the ages of all but that will not be a good option as you will see further how the value changes. 

Lets plot some of the graphs and identify the relationship of age with other features.

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot('Pclass','Age',data=train)

From this we can see that the average age for Pclass 1 is around 38 where as for other classes its 29 and 23 respectively. So just by doing the mean we would have replaced the value to 29.6 which would have not performed as good as this . So lets replace the NaN values with the values we found for each class.

In [ ]:
def Age_1(cols) :
    Age=cols[0]
    Pclass=cols[1] 
    
    if pd.isnull(Age) :
        if Pclass==1 :
            return 38
        elif Pclass==2:
            return 29
        else :
            return 24
    else :
        return Age
                

In [ ]:
train['Age']=train[['Age','Pclass']].apply(Age_1,axis=1)

Here we have replaced the values of Age column with their respective mean values.

Now by seeing the data we can see that Passengerid, Cabin , Ticket, and Name is of no use for determining whether he survived or not .So lets just remove them from our train as well as test dataset.


In [ ]:
train.drop(['PassengerId','Ticket','Cabin','Name'],inplace=True,axis=1)

In [ ]:
X_test.drop(['PassengerId','Ticket','Cabin','Name'],inplace=True,axis=1)

Now there is NaN value in Embarked column and that also only two of them . So Just dropping these two values wont make much difference in our dataset.

In [ ]:
train=train.dropna(axis=0,how='any')

Lets see if our dataset contains any NaN values or not . 

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.isnull(),yticklabels=False,cbar=True)


Here we will also have to remove the NaN values and replace it will other values and repeat the same steps on your X_test dataset also bcz it also contains null values. I have just done that in one cell but you can do it by following previous steps . 

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot('Pclass','Age',data=X_test)


In [ ]:
def Age_1(cols) :
    Age=cols[0]
    Pclass=cols[1] 
    
    if pd.isnull(Age) :
        if Pclass==1 :
            return 42 
        elif Pclass==2:
            return 25
        else :
            return 22
    else :
        return Age
                

In [ ]:
X_test['Age']=X_test[['Age','Pclass']].apply(Age_1,axis=1)

Now 

 Lets start visualising it in depth and get some insight out of it .

In [ ]:
sns.countplot('Survived',hue='Sex',data=train)


This clearly shows that a lot of females survived over males .

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot('Pclass','Fare',data=train)

This tells us that for Pclass 1 they actually paid higher money while other classes didnt pay much money but also Pclass 1 and 3 have a lotmore outliers that would led to overfitting so it must be taken care off.

In [ ]:
sns.countplot('Survived',hue='Pclass',data=train)

Survival rate of Pclass is much more than other but for Pclass 2 is almost the same and so by seeing this we cant just ignore the Fare feature and drop it and it is very crucial for further predictions. 

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(train['Fare'],bins=40)

This doesnt look like a proper gaussian curve and might tend to overfit the curve as it has very less values of people survived for fare greater than 80$.

In [ ]:
train_fare = train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
plt.figure(figsize=(15,5))
sns.distplot(train_fare,bins=40)


Now we can see that the curve it equally distributed and will perfrom well under any classifier.


In [ ]:
fare=train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

Replacing the Fare values by their logerithmic values

Now lets replace the Embarked and Sex columns with their binary values


In [ ]:
sex=pd.get_dummies(train['Sex'],drop_first=True)
emb=pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
y_train=train['Survived']
train.drop(['Embarked','Sex','Survived','Fare'],inplace=True,axis=1)

The dummies are created and now the original dataset we need to concate with the newly created one 

In [ ]:
X_train=pd.concat([train,sex,emb,fare],axis=1)

In [ ]:
y_test.drop('PassengerId',inplace=True,axis=1)

In [ ]:
X_train

This is our final dataset for training the dataset and same goes for y_train .

I have to perform the following steps on X_test i.e. test data as well bcz even it contains some of the data like this 
.Repeat the steps


In [ ]:
test_fare=X_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [ ]:
test_sex=pd.get_dummies(X_test['Sex'],drop_first=True)
test_emb=pd.get_dummies(X_test['Embarked'],drop_first=True)
X_test.drop(['Embarked','Sex','Fare'],inplace=True,axis=1)
X_test=pd.concat([X_test,test_sex,test_emb,test_fare],axis=1)

In [ ]:
Y_test=pd.concat([X_test,test_sex,test_emb,test_fare,y_test],axis=1)
Y_test=Y_test.dropna(axis=0,how='any')


In [ ]:
y_test=Y_test['Survived']

Now we only problm we would face is that our X_test contains 417 rows of data whereas our y_test contains 418 rows of data . So instead initiall while removing the NaN value from fare dataset we would have just used OneHotEncoder for categorical data and IterativeImputer for Numerical data to solve it . But it just one value other way is to just concate it and then drop that value ,  so that it will remove it from our y_test dataset as well.

Now our dataset preprocessing and visualisation is complete and now its time to use the proper method to train and test the dataset to get the best output.

# Using Logistic Regression Algorithm 

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn import metrics 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [ ]:
lgr=LogisticRegression(max_iter=500)
lgr_train=lgr.fit(X_train,y_train)
y_pred = lgr.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(n_estimators=1000,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred=rnd_clf.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

# Using Bagging technique 

Also used GridSearch to use proper hyperparameter and results are much better than using just the logistic regression algorithm.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [ ]:
params = [{'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}]
b_clf=BaggingClassifier(GridSearchCV(DecisionTreeClassifier(random_state=42),params,cv=3,verbose=1),n_estimators=1000,max_samples=100,bootstrap=True,n_jobs=-1)
b_clf.fit(X_train,y_train)
y_pred=b_clf.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
y_submit=pd.DataFrame(y_pred)
y_submit.to_csv("Submission.csv")

In [ ]:
Submission

So for the last technique i.e. bagging I almost got 99.5% accuracy.Though we can use various other techniques such as voting Classifier combining various types of supervised learning algorithm and achieve accuracy may be better or almost equal to this.

Plz feel free to inform if any mistakes are there in this model . If you liked this model plz give a upvote.

# Thank you !